In [1]:
import random
import string
import time

import tensorflow as tf
import numpy as np

from scipy import sparse

def map_nearest_neighbor(string_list, top_args, sim_matrix):
    
    if type(string_list)!=np.array:
        string_list = np.array(string_list)
    
    results = []
    values = []
    for idx, top_arg in enumerate(top_args):
        results.append(string_list[top_arg])
        values.append(sim_matrix[idx].toarray()[0][top_arg])
        
    return np.array(results), np.array(values)



def onehot_string(x, max_length):
    
    if x!=x:
        return None
    
    #Pad to max length
    while len(x)<max_length:
        x+='<'
        
    sparse_matrix = sparse.hstack([sparse.csr_matrix(
        [0 if char_dict[current_char]!=i else 1 for i in range(len(char_dict))]) for current_char in x])
    
    return sparse_matrix

In [2]:
alphabet = string.ascii_lowercase
#alphabet= 'abc'

In [3]:
test_min_str_length = 2
test_max_str_length = 25
num_of_entries = 1000

In [4]:
list_of_strs = [''.join([alphabet[random.randrange(0, len(alphabet))] 
        for i in range(random.randrange(test_min_str_length, test_max_str_length))]) for j in range(num_of_entries)]

In [5]:
list_of_chars = set.union(*[set(i) for i in list_of_strs])

In [6]:
char_dict = {value:idx for idx, value in enumerate(sorted(list_of_chars))}

In [7]:
char_dict['<'] = max(char_dict.values())+1

In [8]:
max_length = max([len(i) for i in list_of_strs])

In [9]:
start_time = time.time()
sparse_strs = sparse.vstack([onehot_string(str_, max_length) for str_ in list_of_strs])
end_time = time.time()

In [10]:
sparse_strs = sparse_strs.tocsr()

In [11]:
print ('time to generate one hot representation:%.3g s' % (end_time-start_time))

time to generate one hot representation:7.44 s


In [12]:
print('size of matrix to operate on:', sparse_strs.shape)

size of matrix to operate on: (1000, 648)


In [13]:
n_tiles = sparse_strs.shape[0]

In [14]:
max_length = sparse_strs.shape[1]

In [15]:
max_str_length = max([len(i) for i in list_of_strs])

In [16]:
tf.reset_default_graph()

In [17]:
ref_sparse = tf.sparse_placeholder(dtype=tf.float32, shape=[sparse_strs.shape[0]]+ list(sparse_strs.shape))
act_sparse = tf.sparse_placeholder(dtype=tf.float32, shape=[sparse_strs.shape[0]]+ list(sparse_strs.shape))

In [18]:
hamming_dists = tf.abs(tf.sparse_add(ref_sparse, act_sparse))

In [19]:
hamming_dist_sum = tf.sparse_reduce_sum_sparse(hamming_dists, axis=2)

In [20]:
#nearest_neighbors = tf.nn.top_k(hamming_dist_sum, k=2, sorted=True)

In [21]:
indices = np.reshape(np.concatenate(sparse_strs.nonzero()), [2, -1])

In [22]:
ref_tile_indices = []
for row in range(sparse_strs.shape[0]):
    a0=time.time()
    np_concat = np.concatenate([[[row for i in range(indices.shape[1])]], indices])
    a1=time.time()
    current_slice = np.transpose(np_concat)
    a2=time.time()
    #print (a2-a1, a1-a0)
    ref_tile_indices.append(current_slice)

In [23]:
ref_tile_indices = np.concatenate(ref_tile_indices)

In [24]:
act_tile_indices = []
for row in range(sparse_strs.shape[0]):
    start_time = time.time()    
    rows, cols = sparse_strs.shape
    current_row = sparse.csr_matrix((np.tile(sparse_strs[row].data, sparse_strs.shape[0]), 
                                     np.tile(sparse_strs[row].indices, sparse_strs.shape[0]),
                           np.arange(0, rows*sparse_strs[row].nnz + 1, sparse_strs[row].nnz)), 
                                    shape=sparse_strs.shape)
    end_time_0 = time.time()
    #current_row = sparse.vstack(t)
    end_time_1 = time.time()
    current_row = np.reshape(np.concatenate(current_row.nonzero()), [2, -1])
    end_time_2 = time.time()
    concate = np.concatenate([[[row for i in range(indices.shape[1])]], current_row])
    end_time_3 = time.time()
    current_slice = np.transpose(concate)
    end_time_4 = time.time()
    #print (end_time_0-start_time, end_time_1-end_time_0, end_time_2-end_time_1, end_time_3-end_time_2)
    act_tile_indices.append(current_slice)

In [25]:
act_tile_indices = np.concatenate(act_tile_indices)

In [26]:
assert act_tile_indices.shape == act_tile_indices.shape

In [31]:
payload_dict = {
               ref_sparse.indices:ref_tile_indices, 
               ref_sparse.values:np.ones(act_tile_indices.shape[0]),
               act_sparse.indices:act_tile_indices, 
               act_sparse.values:-1*np.ones(act_tile_indices.shape[0])
               }

In [32]:
start_time = time.time()
with tf.Session() as sess:
    t = sess.run(hamming_dist_sum, payload_dict)
end_time = time.time()

In [33]:
print('time to process:%.3g s' % (end_time - start_time))

time to process:54.7 s


In [34]:
rows = t.indices[:,0]
cols = t.indices[:,1]
data = t.values

In [36]:
sim_matrix = sparse.csr_matrix((data, (rows, cols)), shape=t.dense_shape)

In [37]:
args = np.argsort(sim_matrix.toarray(), axis=-1)

In [38]:
top_args = args[:,:4]

In [48]:
nn_results = map_nearest_neighbor(list_of_strs, top_args, sim_matrix)